In [1]:
import pandas as pd 
import numpy as np
import time
import requests

### Historical data from the odds api, but i have to pay. Worst case ill pay, if im not able to scrap from betting pros

In [ ]:
import requests
base_url = "https://api.the-odds-api.com/v4/sports/basketball_nba/events/"
API_KEY = '8aa8bb1fc5d56c98b45d6f3f58beab70'
Sport = 'basketball_nba'
Regions = 'us'
Market = 'h2h,odds'
Odds_format = 'american'
Date_format = 'iso' 
date = '2021-10-22T22:45:00Z'
id = '34e829d65b6a0cbb49573338a86772ec'

url = (
    f"https://api.the-odds-api.com/v4/historical/sports/{Sport}/events"
    f"?apiKey={API_KEY}"
    f"&date={date}"
    f"&regions={Regions}"
)

response = requests.get(url)
if response.status_code != 200:
    print(f"Failed to get events: {response.status_code}, {response.text}")
else:
    events = response.json()
    for event in events:
        print(event['id'], event['commence_time'], event['home_team'], event['away_team'])

### Grabs players odds for the day(US all boookmakers, DFS is prizepicks and underdogs)

In [4]:
from NBAPropFinder.NBAPropFinder import NBAPropFinder

nba_props = NBAPropFinder()
prizePicks = nba_props.dataframe
prizePicks.to_csv('PROPS_DATA/Playoffs_US(6_10_25).csv')

Scraping Odds API...
Organizing Data...


### Single Bets from bookmakers that dont include prizePicks or UnderDogs

In [28]:
data = pd.read_csv('PLAYOFF_DATA/PLAYOFFS_25_FEATURES.csv')
all_bookmakers = pd.read_csv('./PROPS_DATA/Playoffs_US(6_10_25).csv')
prop_type = all_bookmakers['CATEGORY'].unique()

In [22]:
import scipy.stats as stats

def single_bet(data,bookmakers,category ='player_points', stat_line = 'PTS'):  
    Props = bookmakers[['NAME', 'BOOKMAKER', 'CATEGORY','LINE','OVER/UNDER', 'ODDS']].loc[bookmakers['CATEGORY'] == category]
    results = []

    for idx, row in Props.iterrows():
        name = row['NAME']
        bookmaker = row['BOOKMAKER']
        category = row['CATEGORY']
        line = row['LINE']
        over_under = row['OVER/UNDER']
        odds = row['ODDS']

        # Filter player data
        df = data.loc[data['PLAYER_NAME'] == name]
        if df.empty:
            continue

        # Mean and std dev of actual results
        mean = round(df[stat_line].mean(), 2)
        std = round(df[stat_line].std(), 2)
        if std == 0:
            continue

        # Z-score-based probability
        z_score = (line - mean) / std
        norm_prob = 1 - stats.norm.cdf(z_score)
        if over_under == "Under":
            norm_prob = 1 - norm_prob

        # Monte Carlo simulation
        sim_points = np.random.normal(mean, std, 10000)
        sim_prob = np.mean(sim_points > line)
        if over_under == "Under":
            sim_prob = 1 - sim_prob

        # Weighted average of sim and norm prob
        final_prob = 0.8 * sim_prob + 0.2 * norm_prob
        
        # EV calculation
        stake = 100
        profit = (odds / 100) * stake if odds > 0 else (100 / abs(odds)) * stake
        payout = stake + profit
        ev = (final_prob * profit) - ((1 - final_prob) * stake)

        # Kelly Criterion
        def kelly_criterion(prob, payout, stake):
            net_profit = payout - stake
            loss_prob = 1 - prob
            kelly = (net_profit * prob - loss_prob * stake) / net_profit
            return max(0, round(kelly, 4))

        kelly = kelly_criterion(final_prob, payout, stake)

        results.append({
            'NAME': name,
            'BOOKMAKER': bookmaker,
            'CATEGORY': category,
            'LINE': line,
            'OVER/UNDER': over_under,
            'ODDS': odds,
            'SIM PROB': round(final_prob, 3),
            'EV': round(ev, 2),
            'KELLY CRITERION': kelly,
        })

    return pd.DataFrame(results).sort_values(by='EV', ascending=False).reset_index(drop=True)

current = single_bet(data, all_bookmakers, category='player_points', stat_line='PTS')
current.head(15)
# current.to_csv('PROPS_EV/singleBets(6_9_25).csv')


,NAME,BOOKMAKER,CATEGORY,LINE,OVER/UNDER,ODDS,SIM PROB,EV,KELLY CRITERION
0,Isaiah Hartenstein,DraftKings,player_points,7.5,Over,114,0.638,36.54,0.3205
1,Isaiah Hartenstein,BetRivers,player_points,7.5,Over,110,0.642,34.88,0.3171
2,Isaiah Hartenstein,Bovada,player_points,8.5,Over,150,0.534,33.61,0.2241
3,T.J. McConnell,Bovada,player_points,9.5,Over,210,0.430,33.21,0.1582
4,Isaiah Hartenstein,BetRivers,player_points,6.5,Over,-124,0.733,32.50,0.4030
5,Isaiah Hartenstein,BetOnline.ag,player_points,6.5,Over,-125,0.731,31.64,0.3955
6,Isaiah Hartenstein,FanDuel,player_points,6.5,Over,-128,0.739,31.59,0.4044
7,Isaiah Hartenstein,Bovada,player_points,9.5,Over,205,0.429,30.94,0.1509
8,Bennedict Mathurin,Bovada,player_points,10.5,Over,165,0.493,30.69,0.1860
9,T.J. McConnell,Bovada,player_points,8.5,Over,155,0.511,30.24,0.1951


## PrizePicks & Underdog
## 2 leg EVs since we cant do single bets on those platforms

In [6]:
data = pd.read_csv('PLAYOFF_DATA/PLAYOFFS_25_FEATURES.csv')
pp_data = pd.read_csv('PROPS_DATA/Playoffs_DFS(6_10_25).csv')
prizePicks = pp_data[['NAME', 'BOOKMAKER', 'CATEGORY','LINE','OVER/UNDER', 'ODDS']].loc[pp_data['CATEGORY'] == 'player_points']
prizePicks

,NAME,BOOKMAKER,CATEGORY,LINE,OVER/UNDER,ODDS
0,Myles Turner,PrizePicks,player_points,14.5,Over,-137
1,Myles Turner,PrizePicks,player_points,14.5,Under,-137
2,T.J. McConnell,PrizePicks,player_points,7.0,Over,-137
3,T.J. McConnell,PrizePicks,player_points,7.0,Under,-137
4,Luguentz Dort,PrizePicks,player_points,8.0,Over,-137
...,...,...,...,...,...,...
94,T.J. McConnell,Underdog,player_points,6.5,Under,-137
95,Thomas Bryant,Underdog,player_points,2.5,Over,-137
96,Thomas Bryant,Underdog,player_points,2.5,Under,-137
97,Obi Toppin,Underdog,player_points,9.5,Over,-137


### Functions used to grab the ev and kelly

In [12]:
import pandas as pd
import numpy as np
from scipy.stats import norm

def zscore_prob(mean, std, line, side="over"):
    z = (line - mean) / std
    prob = 1 - norm.cdf(z)
    return prob if side == "over" else 1 - prob

def kelly_criterion(probability, payout, stake, kelly_fraction=1.0):
    netProfit = payout - stake
    probabilityOfLoss = 1 - probability
    kelly = (netProfit * probability - probabilityOfLoss) / netProfit
    return max(0, round(kelly * kelly_fraction, 4))

def prizePicksProps(data, prizePicks, simulations=10000, stake=100, payout=300, weight=0.8, stat_line='PTS'):
    def monte_carlo_sim(name1, name2, simulations, stake, payout, weight, stat_line):
        df1 = data.loc[data['PLAYER_NAME'] == name1]
        df2 = data.loc[data['PLAYER_NAME'] == name2]

        line1 = prizePicks.loc[prizePicks['NAME'] == name1, 'LINE'].values[0]
        line2 = prizePicks.loc[prizePicks['NAME'] == name2, 'LINE'].values[0]

        mean1, std1 = df1[stat_line].mean(), df1[stat_line].std()
        mean2, std2 = df2[stat_line].mean(), df2[stat_line].std()
        
        if std1 == 0 or std2 == 0:
            return None

        sim1 = np.random.normal(mean1, std1, simulations)
        sim2 = np.random.normal(mean2, std2, simulations)

        over1, under1 = sim1 > line1, sim1 <= line1
        over2, under2 = sim2 > line2, sim2 <= line2

        mc_probs = {
            'OVER/OVER': np.mean(over1 & over2),
            'UNDER/UNDER': np.mean(under1 & under2),
            'OVER/UNDER': np.mean(over1 & under2),
            'UNDER/OVER': np.mean(under1 & over2),
        }

        z_over1 = zscore_prob(mean1, std1, line1, side='over')
        z_under1 = 1 - z_over1
        z_over2 = zscore_prob(mean2, std2, line2, side='over')
        z_under2 = 1 - z_over2

        z_probs = {
            'OVER/OVER': z_over1 * z_over2,
            'UNDER/UNDER': z_under1 * z_under2,
            'OVER/UNDER': z_over1 * z_under2,
            'UNDER/OVER': z_under1 * z_over2,
        }

        probs = {
            k: weight * mc_probs[k] + (1 - weight) * z_probs[k]
            for k in mc_probs
        }

        evs = {k: round((probs[k] * payout) - stake, 2) for k in probs}

        return {
            'PLAYER 1': name1,
            'PLAYER 2': name2,
            'OVER/OVER': {'prob': round(probs['OVER/OVER'], 4), 'ev': evs['OVER/OVER']},
            'UNDER/UNDER': {'prob': round(probs['UNDER/UNDER'], 4), 'ev': evs['UNDER/UNDER']},
            'OVER/UNDER': {'prob': round(probs['OVER/UNDER'], 4), 'ev': evs['OVER/UNDER']},
            'UNDER/OVER': {'prob': round(probs['UNDER/OVER'], 4), 'ev': evs['UNDER/OVER']}
        }

    pairEV = []
    seen = set()

    for name1 in prizePicks['NAME'].unique():
        for name2 in prizePicks['NAME'].unique():
            if name1 == name2:
                continue
            
            name1_sorted, name2_sorted = sorted([name1, name2])
            pair_key = (name1_sorted, name2_sorted)
            if pair_key in seen:
                continue
            seen.add(pair_key)

            sim_result = monte_carlo_sim(name1, name2, simulations, stake, payout, weight, stat_line)
            if sim_result is None:
                continue
            evs = {
                'OVER': sim_result['OVER/OVER']['ev'],
                'UNDER': sim_result['UNDER/UNDER']['ev'],
                'OVER/UNDER': sim_result['OVER/UNDER']['ev'],
                'UNDER/OVER': sim_result['UNDER/OVER']['ev']
            }
            
            probs = {
                'OVER': sim_result['OVER/OVER']['prob'],
                'UNDER': sim_result['UNDER/UNDER']['prob'],
                'OVER/UNDER': sim_result['OVER/UNDER']['prob'],
                'UNDER/OVER': sim_result['UNDER/OVER']['prob']
            }
            
            bestType = max(evs, key=evs.get)
            bestEV = evs[bestType]
            bestProb = probs[bestType]
            
            pairEV.append({
                'PLAYER 1': name1_sorted,
                'PLAYER 1 LINE': prizePicks.loc[prizePicks['NAME'] == name1_sorted, 'LINE'].values[0],
                'PLAYER 2': name2_sorted,
                'PLAYER 2 LINE': prizePicks.loc[prizePicks['NAME'] == name2_sorted, 'LINE'].values[0],
                'TYPE': bestType,
                'EV': round(bestEV, 2),
                'PROBABILITY': round(bestProb, 4),
                'KELLY CRITERION': kelly_criterion(bestProb, payout, stake)
            })
    
    return pd.DataFrame(pairEV).sort_values(by='EV', ascending=False).reset_index(drop=True)


todaysPairEV = prizePicksProps(data, prizePicks)
todaysPairEV.head(15)

,PLAYER 1,PLAYER 1 LINE,PLAYER 2,PLAYER 2 LINE,TYPE,EV,PROBABILITY,KELLY CRITERION
0,Isaiah Hartenstein,7.0,Shai Gilgeous-Alexander,34.0,UNDER/OVER,40.58,0.4686,0.4659
1,Isaiah Hartenstein,7.0,T.J. McConnell,7.0,OVER,29.24,0.4308,0.4280
2,Shai Gilgeous-Alexander,34.0,T.J. McConnell,7.0,OVER/UNDER,28.00,0.4267,0.4238
3,Chet Holmgren,14.5,Isaiah Hartenstein,7.0,OVER,23.10,0.4103,0.4074
4,Ben Sheppard,2.5,Isaiah Hartenstein,7.0,OVER,22.20,0.4073,0.4043
5,Chet Holmgren,14.5,Shai Gilgeous-Alexander,34.0,OVER/UNDER,21.78,0.4059,0.4029
6,Cason Wallace,6.0,Isaiah Hartenstein,7.0,UNDER/OVER,21.69,0.4056,0.4026
7,Andrew Nembhard,11.5,Shai Gilgeous-Alexander,34.0,OVER/UNDER,21.37,0.4046,0.4016
8,Bennedict Mathurin,8.5,Isaiah Hartenstein,7.0,OVER,21.32,0.4044,0.4014
9,Bennedict Mathurin,8.5,Shai Gilgeous-Alexander,34.0,OVER/UNDER,20.41,0.4014,0.3984
